In [1]:
import pandas as pd

In [2]:
df_total = pd.read_excel('./answer_1_1.xlsx',
                         header=0,
                         dtype=str).fillna('')

df_total['MRP_SKU'] = ['US' + sku for sku in df_total['SKU'].tolist()]

df_total

,SKU,仓库,品类,MRP_SKU
0,WS02863,美东仓库,WS,USWS02863
1,WS02632,美西仓库,WS,USWS02632
2,OFA294AEB,美西仓库,OFA,USOFA294AEB
3,OFA583AFB,美西仓库,OFA,USOFA583AFB
4,AIHAAA00,美东仓库,AIH,USAIHAAA00
...,...,...,...,...
28426,GS70140,美南仓库,GS,USGS70140
28427,GS71310,美南仓库,GS,USGS71310
28428,GS68880,美南仓库,GS,USGS68880
28429,GS07168,美南仓库,GS,USGS07168


In [3]:
df_base = pd.read_excel('./2.WEEK2—case题目0428.xlsx',
                        sheet_name='附1_US基础数据',
                        header=0,
                        dtype=str).fillna('')

df_base

,MRP SKU,01-31,02-07,02-14,02-21,02-28,03-07,03-14,03-21,03-28,04-04,04-11,04-18,周销,辅助列
0,USWS02863,22,4,7,14,22,11,25,15,25,21,26,47,23.6,239
1,USWS02632,33,30,30,41,53,65,53,39,31,23,41,32,30.1,471
2,USOFA294AEB,36,43,32,48,31,70,42,4,15,24,23,30,50,398
3,USOFA583AFB,44,68,47,31,8,9,22,22,21,19,29,30,60,350
4,USAIHAAA00,4,2,2,13,9,7,11,26,19,11,17,24,20.4,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28561,USGS70140,0,0,0,0,0,0,0,0,0,0,0,0,2,0
28562,USGS71310,0,0,0,0,0,0,0,0,0,0,0,0,1.5,0
28563,USGS68880,0,0,0,0,0,0,0,0,0,0,0,0,1.5,0
28564,USGS07168,0,0,0,0,0,0,0,0,0,0,0,0,3,0


In [4]:
df_total = pd.merge(left=df_total,
                    right=df_base,
                    how='left',
                    left_on='MRP_SKU',
                    right_on='MRP SKU').fillna('')

df_total = df_total[['MRP_SKU', '品类', '辅助列']]
df_total['辅助列'] = df_total['辅助列'].astype(int)
df_total.columns = ['MRP_SKU', '品类', '销量']

df_total

,MRP_SKU,品类,销量
0,USWS02863,WS,239
1,USWS02632,WS,471
2,USOFA294AEB,OFA,398
3,USOFA583AFB,OFA,350
4,USAIHAAA00,AIH,145
...,...,...,...
28426,USGS70140,GS,0
28427,USGS71310,GS,0
28428,USGS68880,GS,0
28429,USGS07168,GS,0


In [5]:
df_category = df_total[['品类', '销量']]

df_category

,品类,销量
0,WS,239
1,WS,471
2,OFA,398
3,OFA,350
4,AIH,145
...,...,...
28426,GS,0
28427,GS,0
28428,GS,0
28429,GS,0


In [6]:
df_category = df_category.groupby([df_category['品类']], as_index=False).sum()

df_category = df_category.sort_values(by=['销量'], ascending=[False]).reset_index(drop=True)[:10]
df_category['No.'] = [i+1 for i in range(len(df_category))]
df_category = df_category[['No.', '品类', '销量']]

df_category

,No.,品类,销量
0,1,GS,12304
1,2,BC,9110
2,3,FP,8646
3,4,WR,8011
4,5,DLA,5783
5,6,ABS,4794
6,7,OS,4028
7,8,SG,4016
8,9,IC,3751
9,10,PSP,3458


In [7]:
list_df = []
for i in range(10):
    df_temp = df_total[df_total['品类'] == df_category.loc[i, '品类']]
    df_temp = df_temp.sort_values(by=['销量'], ascending=[False]).reset_index(drop=True)[:10]
    df_temp['No.'] = [i+1 for i in range(len(df_temp))]
    df_temp = df_temp[['No.', 'MRP_SKU', '品类', '销量']]
    
    list_df.append(df_temp)
    
df_sku = pd.concat(list_df, ignore_index=True)

df_sku

,No.,MRP_SKU,品类,销量
0,1,USGS06060Z,GS,345
1,2,USGSM3570T,GS,336
2,3,USGS43640B,GS,316
3,4,USGS61560B,GS,304
4,5,USGS44780B,GS,262
...,...,...,...,...
95,6,USPSP15264D,PSP,43
96,7,USPSP15844,PSP,41
97,8,USPSP68740,PSP,38
98,9,USPSP07272,PSP,38


In [8]:
writer = pd.ExcelWriter('./answer_4.xlsx')
df_category.to_excel(writer, sheet_name='Sheet1', index=False)
df_sku.to_excel(writer, sheet_name='Sheet2', index=False)
writer.close()